In [16]:
from transform import preprocessing

df = preprocessing(21)

Data has been written to data/preprocessed/21-round-preprocessed.csv


In [17]:
print(df['구단'].value_counts())

구단
대전      41
김천      40
광주      38
전남      37
천안      35
전북      35
강원      35
수원      35
포항      34
인천      34
서울      34
울산      34
제주      33
대구      32
충남아산    32
성남      32
서울E     32
수원FC    31
경남      31
김포      31
부천      31
안양      31
충북청주    30
부산      29
안산      29
Name: count, dtype: int64


In [18]:
for column in df.columns:
    print(column)

index
선수명
구단
포지션
등번호
출전시간(분)
득점
도움
슈팅
유효 슈팅
차단된슈팅
벗어난슈팅
PA내 슈팅
PA외 슈팅
오프사이드
프리킥
코너킥
스로인
드리블 시도
드리블 성공
드리블 성공%
패스 시도
패스 성공
패스 성공%
키패스
전방 패스 시도
전방 패스 성공
전방 패스 성공%
후방 패스 시도
후방 패스 성공
후방 패스 성공%
횡패스 시도
횡패스 성공
횡패스 성공%
공격지역패스 시도
공격지역패스 성공
공격지역패스 성공%
수비지역패스 시도
수비지역패스 성공
수비지역패스 성공%
중앙지역패스 시도
중앙지역패스 성공
중앙지역패스 성공%
롱패스 시도
롱패스 성공
롱패스 성공%
중거리패스 시도
중거리패스 성공
중거리패스 성공%
숏패스 시도
숏패스 성공
숏패스 성공%
크로스 시도
크로스 성공
크로스 성공%
경합 지상 시도
경합 지상 성공
경합 지상 성공%
경합 공중 시도
경합 공중 성공
경합 공중 성공%
태클 시도
태클 성공
태클 성공%
클리어링
인터셉트
차단
획득
블락
볼미스
파울
피파울
경고
퇴장
xG
득점/xG
Defensive Action
Possession won
득점/90
도움/90
슈팅/90
유효 슈팅/90
차단된슈팅/90
벗어난슈팅/90
PA내 슈팅/90
PA외 슈팅/90
오프사이드/90
프리킥/90
코너킥/90
스로인/90
드리블 시도/90
드리블 성공/90
패스 시도/90
패스 성공/90
키패스/90
전방 패스 시도/90
전방 패스 성공/90
후방 패스 시도/90
후방 패스 성공/90
횡패스 시도/90
횡패스 성공/90
공격지역패스 시도/90
공격지역패스 성공/90
수비지역패스 시도/90
수비지역패스 성공/90
중앙지역패스 시도/90
중앙지역패스 성공/90
롱패스 시도/90
롱패스 성공/90
중거리패스 시도/90
중거리패스 성공/90
숏패스 시도/90
숏패스 성공/90
크로스 시도/90
크로스 성공/90
경합 지상 시도/90
경합 지상 성공/90
경합 공중 시도/90
경합 공중 성공/90
태클 시도/90
태클 성공/90
클리어링/90
인터셉트/9

In [98]:
import numpy as np
from scipy.stats import rankdata
import pandas as pd
import plotly.graph_objects as go

class RadarChart(object):
    def __init__(self, player_name, df):
        self.fig = go.Figure()
        self.df = df
        self.player_name = player_name
        self.player_position = self.df[df['선수명'] == player_name]["포지션"].values[0]
        self.string_column = ['index', '선수명', '구단', '포지션', '등번호']
        self.radar_columns = ['득점/xG', '슈팅/90', '유효 슈팅/90', '드리블 시도/90', '공격지역패스 성공%', '키패스/90', 'Defensive Action/90', 'Possession won/90', '경합 공중 성공/90']

        # 특정 선수의 데이터프레임 필터링
        self.relative_df = df[df['포지션'] == self.player_position]
        self.relative_df = self.relative_df[self.relative_df['출전시간(분)'] >= 450]
        
        # 퍼센타일 스케일링
        self.radar_scaled_df = self.relative_df.copy()
        self.radar_scaled_columns = self.relative_df[self.radar_columns]
        
        def percentile_rank(x):
            # 모든 값이 동일한 경우를 처리
            if len(np.unique(x)) == 1:
                return np.zeros(len(x))
            ranks = rankdata(x)
            scaled = 100 * (ranks - 1) / (len(ranks) - 1)
            return scaled
        
        self.radar_scaled_df[self.radar_columns] = self.radar_scaled_columns.apply(percentile_rank)
        
        # 특정 선수의 데이터 추출
        self.target_df = self.radar_scaled_df[self.radar_scaled_df["선수명"] == player_name][self.radar_columns]
        self.key_value = '키패스/90'
        print(self.radar_scaled_df.sort_values(by=self.key_value, ascending=False).head(10))

    def plot(self):
        if self.target_df.empty:
            print(f"No data available for player {self.player_name}")
            return

        categories = self.radar_columns
        values = self.target_df.values.flatten().tolist()

        # Close the loop
        values += values[:1]
        categories += categories[:1]

        self.fig.add_trace(go.Scatterpolar(
            r=values,
            theta=categories,
            fill='toself',
            name=self.player_name
        ))

        self.fig.update_layout(
            polar=dict(
                radialaxis=dict(
                    range=[0, 100],
                ),
                angularaxis=dict(
                    showgrid=False,
                )
            ),
            plot_bgcolor='rgba(0, 0, 0, 0)',
            paper_bgcolor='rgba(0, 0, 0, 0)'
        )

        self.fig.show()
        self.fig.write_image(f"output/{self.player_name}-radar.png", format='png')
        



In [99]:
mh_radar = RadarChart('양민혁', df)
mh_radar.plot()


     index   선수명    구단 포지션   등번호  출전시간(분)   득점   도움    슈팅  유효 슈팅  ...  \
503  498.0   이동경    울산  MF  14.0    611.0  7.0  5.0  24.0   14.0  ...   
214  212.0   라마스    부산  MF  10.0   1866.0  4.0  4.0  76.0   23.0  ...   
747  740.0   최우진    인천  MF  14.0   1028.0  0.0  3.0   7.0    2.0  ...   
778  771.0    페신    부산  MF   7.0   1596.0  4.0  4.0  33.0   13.0  ...   
766  759.0   카즈키    수원  MF  14.0    805.0  0.0  2.0  11.0    4.0  ...   
124  123.0   김영욱   서울E  MF  14.0    497.0  0.0  0.0   2.0    0.0  ...   
216  214.0   로페즈    부산  MF  11.0   1138.0  0.0  2.0  34.0   15.0  ...   
393  388.0  안데르손  수원FC  MF  70.0   1815.0  1.0  8.0  50.0   14.0  ...   
229  227.0  마테우스    안양  MF   7.0   1642.0  4.0  4.0  50.0   24.0  ...   
362  357.0   송민규    전북  MF  10.0   1445.0  4.0  6.0  19.0    7.0  ...   

        획득/90     블락/90    볼미스/90     파울/90    피파울/90     경고/90     퇴장/90  \
503  5.302782  1.325696  0.883797  1.031097  0.589198  0.000000  0.000000   
214  8.344051  0.868167  0.723473  0.53054

In [71]:
import plotly.express as px

class ScatterChart(object):
    def __init__(self, x, y, player_name, df):
        self.player_name = player_name
        self.player_position = df[df['선수명'] == player_name]['포지션'].values[0]
        self.df = df[(df['출전시간(분)'] >= 1000) & (df['포지션'] == self.player_position)]
        self.x = x
        self.y = y
        
        # Calculate the mean values for x and y axes
        self.mean_x = self.df[self.x].mean()
        self.mean_y = self.df[self.y].mean()

        # Create the scatter plot
        self.fig = px.scatter(
            self.df,
            x=self.x,
            y=self.y,
            hover_name='선수명', # Set player names to appear on hover
            color_discrete_sequence=['lightgray'],  # Set the default color to lightgray
        )
        
        # Highlight the specific player
        self.highlight_player()
        
        # Add mean lines
        self.add_mean_lines()

    def highlight_player(self):
        # Add a trace for the specific player to highlight
        player_data = self.df[self.df['선수명'] == self.player_name]
        if not player_data.empty:
            self.fig.add_scatter(
                x=player_data[self.x],
                y=player_data[self.y],
                mode='markers',
                marker=dict(size=15, color='white', symbol='circle',
                            line=dict(color='lightgray', width=2)),
                name=self.player_name,
                hoverinfo='name'
            )

    def add_mean_lines(self):
        # Add vertical and horizontal lines for the mean values
        self.fig.add_shape(
            type="line",
            x0=self.mean_x,
            y0=self.df[self.y].min(),
            x1=self.mean_x,
            y1=self.df[self.y].max(),
            line=dict(color="gray", width=1),
        )
        self.fig.add_shape(
            type="line",
            x0=self.df[self.x].min(),
            y0=self.mean_y,
            x1=self.df[self.x].max(),
            y1=self.mean_y,
            line=dict(color="gray", width=1),
        )

    def plot(self):
        self.fig.update_layout(
            plot_bgcolor='rgba(0,0,0,0)',  # Transparent plot background
            paper_bgcolor='rgba(0,0,0,0)',  # Transparent paper background
            xaxis=dict(showgrid=False),  # Hide x-axis grid
            yaxis=dict(showgrid=False),  # Hide y-axis grid
        )
        # Replace invalid characters in filename
        safe_x = self.x.replace("/", "_")
        safe_y = self.y.replace("/", "_")
        filename = f'output/{self.player_name}_scatter_{safe_x}_{safe_y}.png'
        self.fig.write_image(filename, format='png')
        self.fig.show()

# Usage
# Assuming you have a DataFrame 'df' and you want to create a scatter plot for a player
# scatter_chart = ScatterChart(x='출전시간(분)', y='골', player_name='양민혁', df=df)
# scatter_chart.plot()


In [73]:
for column in df.columns:
    print(column)

index
선수명
구단
포지션
등번호
출전시간(분)
득점
도움
슈팅
유효 슈팅
차단된슈팅
벗어난슈팅
PA내 슈팅
PA외 슈팅
오프사이드
프리킥
코너킥
스로인
드리블 시도
드리블 성공
드리블 성공%
패스 시도
패스 성공
패스 성공%
키패스
전방 패스 시도
전방 패스 성공
전방 패스 성공%
후방 패스 시도
후방 패스 성공
후방 패스 성공%
횡패스 시도
횡패스 성공
횡패스 성공%
공격지역패스 시도
공격지역패스 성공
공격지역패스 성공%
수비지역패스 시도
수비지역패스 성공
수비지역패스 성공%
중앙지역패스 시도
중앙지역패스 성공
중앙지역패스 성공%
롱패스 시도
롱패스 성공
롱패스 성공%
중거리패스 시도
중거리패스 성공
중거리패스 성공%
숏패스 시도
숏패스 성공
숏패스 성공%
크로스 시도
크로스 성공
크로스 성공%
경합 지상 시도
경합 지상 성공
경합 지상 성공%
경합 공중 시도
경합 공중 성공
경합 공중 성공%
태클 시도
태클 성공
태클 성공%
클리어링
인터셉트
차단
획득
블락
볼미스
파울
피파울
경고
퇴장
xG
득점/xG
Defensive Action
Possession won
득점/90
도움/90
슈팅/90
유효 슈팅/90
차단된슈팅/90
벗어난슈팅/90
PA내 슈팅/90
PA외 슈팅/90
오프사이드/90
프리킥/90
코너킥/90
스로인/90
드리블 시도/90
드리블 성공/90
패스 시도/90
패스 성공/90
키패스/90
전방 패스 시도/90
전방 패스 성공/90
후방 패스 시도/90
후방 패스 성공/90
횡패스 시도/90
횡패스 성공/90
공격지역패스 시도/90
공격지역패스 성공/90
수비지역패스 시도/90
수비지역패스 성공/90
중앙지역패스 시도/90
중앙지역패스 성공/90
롱패스 시도/90
롱패스 성공/90
중거리패스 시도/90
중거리패스 성공/90
숏패스 시도/90
숏패스 성공/90
크로스 시도/90
크로스 성공/90
경합 지상 시도/90
경합 지상 성공/90
경합 공중 시도/90
경합 공중 성공/90
태클 시도/90
태클 성공/90
클리어링/90
인터셉트/9

In [77]:
scatter = ScatterChart('슈팅/90', '득점', '양민혁', df)
scatter.plot()

In [23]:
print(df.shape)

(836, 134)
